<a href="https://www.kaggle.com/code/thiagopereiradelima/titanic-disaster-tutorial?scriptVersionId=121406775" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Since this is my first notebook in Kaggle, I've followed the tutorial for learning purpose. After, I've made some tweaks for the Random Forest yet.

## Import libraries and database

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))       

        
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Original survivability

In [4]:
women = train_data.loc[train_data.Sex == 'female']['Survived']
rate_women = sum(women)/len(women)

print('{:.2f}% of women who survived!'.format(rate_women*100))

74.20% of women who survived!


In [5]:
men = train_data.loc[train_data.Sex == 'male']['Survived']
rate_men = sum(men)/len(men)

print('{:.2f}% of men who survived...'.format(rate_men*100))

18.89% of men who survived...


## Creating a RandomForest model

In [6]:
y = train_data['Survived']
features = ['Pclass', 'Sex', 'SibSp', 'Parch']

# change text features to binary
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

#### Defining a function for the submission file

In [7]:
def save_submission(test_data, predictions):
    output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
    output.to_csv('submission.csv', index=False)
    print('Your submission was successfully saved.')

In [8]:
# save_submission(test_data, predictions)

## Simple tweaks
I've just added few simple steps to improve and see a "relative accuracy" without need to commit the file to Kaggle.

#### Getting out the missing values

In [9]:
train_data.drop(columns='Cabin', axis=0, inplace=True)
train_data.fillna(method='ffill', inplace=True)

#### Cheking the accuracy for RandomForest

In [10]:
def train_model(X, y, model):
    '''This is a function that get the X and y, split in train and test data, 
    train the model and check their accuracy. It is just for testing'''    
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify = y)    
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    accuracy_rate = accuracy_score(y_test, predictions) * 100    
    
    return int(accuracy_rate)

In [11]:
y = train_data['Survived']
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Parch']

X = pd.get_dummies(train_data[features])

model = RandomForestClassifier(n_estimators=150, max_depth=5, random_state=2)

In [12]:
limit = 100
accuracies = []
for i in range(0, limit):
    accuracies.append(train_model(X, y, model))    
    
print('The mean accuracy for this setting is {}'.format(sum(accuracies)/limit))

The mean accuracy for this setting is 80.95


### Testing the new setup and making a second submission file
##### Handling with missing values in test_data

In [13]:
test_data.fillna(method='ffill', inplace=True)

Trainning with the second model setup

In [14]:
# X, y and the model remains the same above

X_test = pd.get_dummies(test_data[features])
model.fit(X, y)
predictions = model.predict(X_test)

Saving the submission file.

In [15]:
save_submission(test_data, predictions)

Your submission was successfully saved.


I know there's a lot to improve. Here we can see advanced users just use XGBoost or others advanced algorithms. But, with time, beginners (like me) can learn, compreehend and improve even the simplest model.